Removing Images and labels of pre

In [11]:
import os
import glob

def delete_images_with_keyword(folder_path, keyword):
    # Get a list of image files in the folder
    image_files = glob.glob(os.path.join(folder_path, '*.jpg')) + glob.glob(os.path.join(folder_path, '*.png'))

    # Iterate through each image file
    for image_file in image_files:
        # Check if the keyword is in the file name
        if keyword in os.path.basename(image_file):
            # Delete the image file
            os.remove(image_file)
            print(f"Deleted {image_file}")

# Replace 'folder_path' with the path to your folder containing images
folder_path = 'train4/images'
# Replace 'pre' with your keyword
keyword = 'pre'

# Call the function to delete images with the specified keyword
delete_images_with_keyword(folder_path, keyword)

# Replace 'folder_path' with the path to your folder containing images
folder_path = 'train4/labels'
delete_images_with_keyword(folder_path, keyword)


## Subtract from all images

## Main SubData Images & Label Gen Code

In [12]:
import os, shutil
from PIL import Image
import cv2

train_folder = 'train4'
#####
lbl_list = os.listdir(train_folder+'/labels')
#####
def subtract_binary_images(image1_path, image2_path, output_path):
    image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)
    if image1.shape != image2.shape:
        print("Error: Images must have the same dimensions.")
        return
    result = cv2.subtract(image1, image2)
    cv2.imwrite(output_path, result)
    print(f"Subtracted images and saved result to '{output_path}'")

def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
    else:
        print(f"Directory '{directory}' already exists. Skipping creation.")

def create_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Deleted existing folder: {folder_path}")
    os.makedirs(folder_path)
    print(f"Created folder: {folder_path}")

def convert_png_to_jpg(png_path, jpg_path):
    png_image = Image.open(png_path)
    if png_image.mode in ('RGBA', 'LA'):
        png_image = png_image.convert('RGB')
    png_image.save(jpg_path, format='JPEG', quality=100)

def create_empty_binary_mask(width, height):
    # Create a new black image (mode '1' for binary)
    # Create a new black image (mode 'L' for 8-bit grayscale)
    mask = Image.new('L', (width, height), color=0)
    return mask


lbl_list = os.listdir(train_folder+'/labels')

for i in range(len(lbl_list)):

    tmp_lbl = lbl_list.pop(0)

    create_folder(train_folder+'/subdata'+str(i)+'/train')

    for label_name in lbl_list:
        
        lbl1_path = train_folder+'/labels/'+label_name
        lbl2_path = train_folder+'/labels/'+tmp_lbl
        output_path = train_folder+'/subdata'+str(i)+'/train/'+'sd_'+str(i)+'_'+label_name[:-9]+'_mask.png'

        subtract_binary_images(lbl1_path, lbl2_path, output_path)

        image_png = train_folder+'/images/'+label_name
        image_jpg = train_folder+'/subdata'+str(i)+'/train/'+'sd_'+str(i)+'_'+label_name[:-9]+'.jpg'

        convert_png_to_jpg(image_png,image_jpg)


    tmp_img_png = train_folder+'/images/'+tmp_lbl
    tmp_img_jpg = train_folder+'/subdata'+str(i)+'/train/'+'000.jpg'
    convert_png_to_jpg(tmp_img_png,tmp_img_jpg)

    binary_mask = create_empty_binary_mask(1024, 1024)
    binary_mask.save(train_folder+'/subdata'+str(i)+'/train/'+'000_mask.png')



Created folder: train4/subdata0/train
Subtracted images and saved result to 'train4/subdata0/train/sd_0_bolari_1709_patch_1_1_png_rf_e1954174c34a6ee69ee8c7073d894e36_mask.png'
Subtracted images and saved result to 'train4/subdata0/train/sd_0_bolari_1802_patch_1_1_png_rf_81647124bd8c565e704f14428d42e0fc_mask.png'
Subtracted images and saved result to 'train4/subdata0/train/sd_0_bolari_1812_patch_1_1_png_rf_c3e8517182202e36f3709e7e68a12ae9_mask.png'
Subtracted images and saved result to 'train4/subdata0/train/sd_0_bolari_1902_patch_1_1_png_rf_53ee81544706e462fa08cc228841479b_mask.png'
Subtracted images and saved result to 'train4/subdata0/train/sd_0_bolari_1906_patch_1_1_png_rf_9fa7a2caaa9a4b2d8392ba68c11a0ea5_mask.png'
Subtracted images and saved result to 'train4/subdata0/train/sd_0_bolari_1912_patch_1_1_png_rf_20ec8e4456d5a1b4be195142d9634e81_mask.png'
Subtracted images and saved result to 'train4/subdata0/train/sd_0_bolari_2012_patch_1_1_png_rf_85d2b849e62c8f1c6578fcadce3fe6b7_mask.p

## Now it time for ... One Data Gen

In [68]:
!cd "C:/Users/Asus/Documents/GitHub/one-cd-data-renamer/data_self_gen/train3/subdata3/"

In [69]:
pwd

'C:\\Users\\Asus'

In [27]:
exec(open("C:/Users/Asus/Documents/GitHub/one-cd-data-renamer/data_self_gen/train3/subdata3/one_data_pre_process.py").read())

ok
Renamed: train\000.jpg -> train\000.jpg
Renamed: train\000_mask.png -> train\000_mask.png
Renamed: train\sd_3_park_view_city_0202_patch_1_0_png_rf_add170b8bc0fa3373329a336a0da06b3.jpg -> train\sd_3_park_view_city_0202_patch_1_0_png_rf_add170b8bc0fa3373329a336a0da06b3.jpg
Renamed: train\sd_3_park_view_city_0202_patch_1_0_png_rf_add170b8bc0fa3373329a336a0da06b3_mask.png -> train\sd_3_park_view_city_0202_patch_1_0_png_rf_add170b8bc0fa3373329a336a0da06b3_mask.png
Renamed: train\sd_3_park_view_city_1212_patch_1_0_png_rf_40bfbb62b410d86bff5cddd470605d3e.jpg -> train\sd_3_park_view_city_1212_patch_1_0_png_rf_40bfbb62b410d86bff5cddd470605d3e.jpg
Renamed: train\sd_3_park_view_city_1212_patch_1_0_png_rf_40bfbb62b410d86bff5cddd470605d3e_mask.png -> train\sd_3_park_view_city_1212_patch_1_0_png_rf_40bfbb62b410d86bff5cddd470605d3e_mask.png
Renamed: train\sd_3_park_view_city_1607_patch_1_0_png_rf_3ca02fad7c46480ac2e0609769388250.jpg -> train\sd_3_park_view_city_1607_patch_1_0_png_rf_3ca02fad7c4648

In [95]:
cd "C:/Users/Asus/Documents/GitHub/one-cd-data-renamer"

C:\Users\Asus\Documents\GitHub\one-cd-data-renamer


In [96]:
pwd

'C:\\Users\\Asus\\Documents\\GitHub\\one-cd-data-renamer'

In [98]:
import subprocess

# Path to the Python file you want to run
cd_dir = "C:/Users/Asus/Documents/GitHub/one-cd-data-renamer/data_self_gen/train3/subdata5/"

# Run the Python file
subprocess.run(['cd', cd_dir])

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [110]:
cd "C:/Users/Asus/Documents/GitHub/one-cd-data-renamer/data_self_gen/train3/"

C:\Users\Asus\Documents\GitHub\one-cd-data-renamer\data_self_gen\train3


c:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [111]:
pwd

'C:\\Users\\Asus\\Documents\\GitHub\\one-cd-data-renamer\\data_self_gen\\train3'

In [105]:
import subprocess

# Path to the Python file you want to run
python_file_path = "C:/Users/Asus/Documents/GitHub/one-cd-data-renamer/data_self_gen/train3/subdata4/one_data_pre_process.py"

# Run the Python file
subprocess.run(["python", python_file_path])


CompletedProcess(args=['python', 'C:/Users/Asus/Documents/GitHub/one-cd-data-renamer/data_self_gen/train3/subdata4/one_data_pre_process.py'], returncode=0)

In [112]:
import subprocess

# Path to the Python file you want to run
python_file_path = "C:/Users/Asus/Documents/GitHub/one-cd-data-renamer/data_self_gen/train3/subdata4/one_data_pre_process.py"

# Run the Python file
subprocess.run(["python", python_file_path])

CompletedProcess(args=['python', 'C:/Users/Asus/Documents/GitHub/one-cd-data-renamer/data_self_gen/train3/subdata4/one_data_pre_process.py'], returncode=1)